# Getting the Git

In [1]:
import os
%matplotlib inline
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib as mpl
import time
from time import sleep
import sys
os.environ['PATH'] += ";C\Program Files\Git\\bin"

'''# ---------------- Old Toolbar Updated -------------------------#
toolbar_width = 40

# setup toolbar
sys.stdout.write("[%s]" % (" " * toolbar_width))
sys.stdout.flush()
sys.stdout.write("\b" * (toolbar_width+1)) 
'''

from time import sleep
import sys

def progbar(i,j):
    sys.stdout.write('\r')
    k = round((i / j) * 100,2)
    sys.stdout.write("%d%%" % k)
    sys.stdout.flush()
    sleep(0.01)

In [76]:
#%%bash
#rm -rf fec-data
#git clone git@github.com:jbisbee1/fec-data.git

In [2]:
%%bash
git log

commit 0549573e24b8e24eaf542ef52cbbe2f8d14204de
Author: jbisbee1 <jimbisbee@yahoo.com>
Date:   Fri Jul 22 15:40:43 2016 -0400

    Updated Qualtrics import .txt file with earmarks firms.

commit fd0054cc07e492fb2f0aa3389652e3f5f8635dd5
Author: jbisbee1 <jimbisbee@yahoo.com>
Date:   Fri Jul 22 09:27:10 2016 -0400

    Implementing fuzzy wuzzy and using it to create Qualtrics surveys for manual checking.

commit 4cd451b67957990f506074a7f38e200ceb053862
Author: jbisbee1 <jimbisbee@yahoo.com>
Date:   Thu Jul 21 16:37:22 2016 -0400

    Updated the readme.md file to remind myself of fuzzyfuzzy.

commit 3f7beb7944abf2f86654fc6b9c15fcfdb61f4091
Author: jbisbee1 <jimbisbee@yahoo.com>
Date:   Thu Jul 21 12:15:12 2016 -0400

    Updated the readme.md file with a brief description of the purpose for this repository.


# Opening the full data

In [3]:
recipients = pd.read_stata("../../rawrecipients.dta")
recipients.head()

,raw_recipient,n
0,! A+ TUTOR U LEARNIGN CENTERS OF CENTRAL FLORI...,1
1,!1T Family Dental,3
2,!2 Technologies,1
3,"""33"" Ranch",1
4,"""Concept"" Employees Of",1


# Fuzzy matching on firm names

I use the fuzzywuzzy package and run through each firm name, comparing it to the remaining 5.6 million in the dataset. 

In [4]:
#%%bash
#pip install fuzzywuzzy

In [5]:
#%%bash
#pip install "C:/Users/jhb362/Dropbox/Programs/python_Levenshtein-0.12.0-cp35-none-win_amd64.whl"

In [4]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [5]:
recclean = recipients['raw_recipient']
recclean.head()

0    ! A+ TUTOR U LEARNIGN CENTERS OF CENTRAL FLORI...
1                                    !1T Family Dental
2                                      !2 Technologies
3                                           "33" Ranch
4                               "Concept" Employees Of
Name: raw_recipient, dtype: object

Each of the firm names is compared against a master list which is stripped of any non-string types (although I'm not sure why any would be in there, there seem to be a few.) I also drop duplicates but, again, I don't think there should be too many in there, given all the work already conducted using Stata.

In [7]:
uniques = recclean.drop_duplicates()
select = [isinstance(e,str) for e in uniques]
uniques2 = uniques[select]

In [8]:
select = [isinstance(e,str) for e in recclean]
recclean2 = recclean[select]
print(len(recclean2), len(uniques2))

5647972 5647972


Confirmed that there are no duplicates in the list. 

Now we're off to the races. This is going to take a goddamned long time so smoke 'em if you got 'em. We go through every firm name in the full list of 5.6m and compare the string to all of the others using the fuzzywuzzy method "process". 

Whoa...too long. I'm going to try and run this on the cluster. No sense in gumming up my machine.

# Write a .txt file in the format for importing to Qualtrics

The code below can be converted to illustrate the process by which I create the .txt files that are formatted to be imported into Qualtrics. I don't run the code here though because it would take a long time.

In [63]:
max = round(len(mask) % 100,0)
start = 0
for z in range(1,max):
    end = z*100
    submask = mask[start:z*100]
    start = z*100+1
    survey_file = open("survey-test-%s" %z +".txt","w")
    survey_file.write("[[AdvancedFormat]] \n\n")
    counter = 0
    for j in range(0,len(submask),10):
        survey_file.write("[[Block: Firm Names - %s" % counter + "]]\n")
        for k in range(0,len(submask[j:j+10])):
            survey_file.write("[[Question:MC:MultipleAnswer:Vertical]] \n \
            %s" % submask[k][0].upper() + "\n\n")
            survey_file.write("[[AdvancedChoices]] \n")
            for i in range(1,len(submask[k][1])):
                survey_file.write("[[Choice: %s" % i +"]] \n \
                %s" % submask[k][1][i][0].upper() +'\n')
            survey_file.write("\n")
        counter +=1
    survey_file.close()
    
    # update the bar
    sys.stdout.write("-")
    sys.stdout.flush()

sys.stdout.write("\n")

----------------------------------------------------------------------------------------


In [22]:
%%bash
git add fec-string-prep.ipynb
git add survey-test.txt
git status
git commit -m "Wrote progress bar % tracker and merged firm names for lobbying, earmarks, grants, contracts, and FEC. Goddamned FEC slows the bastard to a crawl. I also didn't manage to get anything done on the legal dataset which, unfortunately, lists defendents together in a single cell, separated by commas. Stupidly, this dataset also uses the LNAME, FNAME format for individual defendents, making extraction a real bear."

On branch master
Your branch is up-to-date with 'origin/master'.
Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   fec-string-prep.ipynb
	new file:   survey-test.txt

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.ipynb_checkpoints/

[master 0549573] Updated Qualtrics import .txt file with earmarks firms.
 2 files changed, 6352 insertions(+), 227 deletions(-)
 create mode 100644 survey-test.txt


The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
